In [522]:
!pip install selenium

In [523]:
# This Section imports aree the libraries that will be used in the code

# selenium==4.16.0

from selenium import webdriver # to open a web browser
from selenium.webdriver.common.keys import Keys # to be able to type in the browser
from selenium.webdriver.common.by import By # to be able to search things in the browser
from selenium.webdriver.firefox.service import Service as FirefoxService
from webdriver_manager.firefox import GeckoDriverManager


from bs4 import BeautifulSoup # to be able to search the HTML in browser 
import time # to be able to wait for x amount of time seconds. 


linkedin_email =  "chetnagirish2@gmail.com"
linkedin_password = "Boss1234$"

url = 'https://www.linkedin.com/jobs'

In [524]:
# This section contains the code to scrape the data. specifically the salary.

import re

def check_element (elem):
    if elem is None: 
        return ''
    if len (elem) > 0:
        return elem.text.strip()
    else:
        return ''



def extract_hourly_or_salary_range(salary_string):
    hourly_match = re.search(r'\$([\d,]+(?:\.\d+)?)\s?(?:-|to)?\s?\$?([\d,]+(?:\.\d+)?)?\s?\/\s?hr', salary_string)
    salary_match = re.search(r'\$([\d,]+(?:\.\d+)?)\s?K?\s?\/\s?yr\s?(?:-|to)?\s?\$?([\d,]+(?:\.\d+)?)?\s?K?\s?\/\s?yr', salary_string)

    if hourly_match:
        hourly_rate_lower = float(hourly_match.group(1).replace(',', ''))
        hourly_rate_upper = float(hourly_match.group(2).replace(',', '')) if hourly_match.group(2) else hourly_rate_lower

        # Assuming 40 hours per week and 52 weeks per year
        annual_lower = hourly_rate_lower * 40 * 52
        annual_upper = hourly_rate_upper * 40 * 52

        average = (annual_lower + annual_upper) / 2

        return [annual_lower, annual_upper, average]
    elif salary_match:
        salary_lower = float(salary_match.group(1).replace(',', '')) * 1000
        salary_upper = float(salary_match.group(2).replace(',', '')) * 1000 if salary_match.group(2) else salary_lower

        average = (salary_lower + salary_upper) / 2

        return [salary_lower, salary_upper, average]
    else:
        return [0,0,0]

# # Example usage
# strings = [
#     '$57/hr - $78/hr · Vision, +1 benefit',
#     '401(k), +6 benefits',
#     '$117K/yr - $134.2K/yr · Medical benefit',
#     '$55.1K/yr - $68.9K/yr',
#     '$70/hr - $75/hr · 401(k), +1 benefit',
#     'Starting at $40/hr',
#     '$57/hr - $78/hr · Vision, +1 benefit'
# ]

# for salary_string in strings:
#     result = extract_hourly_or_salary_range(salary_string)
#     if result:
#         print(f"Input: {salary_string}\nConverted Range: {result}\n")
#     else:
#         print(f"Input: {salary_string}\nUnable to extract hourly rate or salary range.\n")


In [525]:
#  THis section of the code is to start the browser and go the the respective URL. and located the box for user id and password
browser = webdriver.Firefox(service=FirefoxService(GeckoDriverManager().install()))# start the browser 
browser.get(url)  # in browser search the URL 

email_input = browser.find_element("name", "session_key")
password_input = browser.find_element("name", "session_password")
login_button = browser.find_element("xpath", "//button[@type='submit']")

In [526]:
# This section of the code is to enter the user id and password
email_input.send_keys(linkedin_email)
password_input.send_keys(linkedin_password)

In [527]:
# Click the login button
login_button.click()
time.sleep(5)

In [528]:
# Plug in Job title in the search box
search_box = browser.find_element(By.CLASS_NAME, 'jobs-search-box__text-input')
search_box.send_keys('Data Analyst')
search_box.send_keys(Keys.RETURN)


In [529]:
# locate and clik the date filter
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC

# Wait for the element to be visible
WebDriverWait(browser, 10).until(EC.visibility_of_element_located((By.ID, "searchFilter_timePostedRange")))

date_filter = browser.find_element(By.ID, "searchFilter_timePostedRange")
date_filter.click()


In [530]:
# locate week filter on the date filter

class_name = "t-14 t-black--light t-normal"
span_text = "week"

past_week_x_path= f"//span[@class='{class_name}' and contains(.,'{span_text}')]"
past_week = browser.find_element("xpath", past_week_x_path)
past_week.click()


In [531]:
# click the show results button the the date filter
class_name = "t-14 t-black--light t-normal"
span_text = "week"

past_week_x_path= f"//span[@class='{class_name}' and contains(.,'{span_text}')]"
past_week = browser.find_element("xpath", past_week_x_path)
past_week.click()


In [532]:
# find all the job postings on the wbsite 
jobs_block = browser.find_element(By.CSS_SELECTOR, '.scaffold-layout__list-container')
jobs_list= jobs_block.find_elements(By.CSS_SELECTOR, '.jobs-search-results__list-item')


In [533]:
# scroll down the page
# NEXT STEP : Find the most efficient way to scroll down the list so all the job profiles are loaded. 
x = 0
while (x<=len(jobs_list)-1):
        # Print job element


    browser.execute_script("arguments[0].scrollIntoView();", jobs_list[x])
    x= x+1
    time.sleep(1)
first_job_element = jobs_list[0]

# Print job details
print("First job details:", first_job_element.text)

    

# Ignore the bottom of this section 
# browser.execute_script("arguments[0].scrollIntoView();", jobs_list[len(jobs_list)-1])


First job details: Data Analyst 
Zolon Tech Inc.
Cupertino, CA (Hybrid)
Viewed
Promoted
Easy Apply


In [534]:
# Parse the HTML in 
from bs4 import BeautifulSoup
soup = BeautifulSoup(browser.page_source, "html.parser")

In [535]:
# find all the LI elements in the HTML
target_class = 'jobs-search-results'
target_li_elements = soup.find_all('li', class_=lambda x: x and 'jobs-search-results__list-item' in x)
# target_li_elements = soup.find_all('li', class_=lambda x: x and 'jobs-search-results__list-item' in x)

In [536]:
# Testing the code to see if LI element truely has the data we need to extract 
#  NEXT STEP: Write a loop the iterate through all the LI elements and check if the data we need is in the LI element. Append the date in to an array
#  Import the data in to dataframe. 

num=15


job_title = check_element (target_li_elements[num].find('a', class_=lambda x: x and 'title' in x))
company = check_element ( target_li_elements[num].find('span', class_= 'job-card-container__primary-description') )
location = check_element (target_li_elements[num].find('div', class_= 'artdeco-entity-lockup__caption ember-view') )
salary = check_element (target_li_elements[num].find('div', class_= lambda x: x and 'mt1 t-sans t-12 t-black--light' in x) )
link = target_li_elements[num].find('a').get('href')

dic = {     'job_title': job_title ,
            'company': company,
            'location': location,
            'salary_string': salary,
            'lower_bound_extracted_salary':  (extract_hourly_or_salary_range(salary)[0]),
            'upper_extracted_salary':  (extract_hourly_or_salary_range(salary)[1]),
            'average_extracted_salary':  (extract_hourly_or_salary_range(salary)[2]),
            'link': 'www.linkedin.com'+link
       
          }

print(dic)

{'job_title': 'Business Data Analyst', 'company': 'Indotronix Avani Group', 'location': 'St Petersburg, FL (On-site)', 'salary_string': '', 'lower_bound_extracted_salary': 0, 'upper_extracted_salary': 0, 'average_extracted_salary': 0, 'link': 'www.linkedin.com/jobs/view/3834873605/?eBP=CwEAAAGOZHBY-Ads0IBdDw-Eh7frY94G1_WuqglxIG-EJV2SjZ5fR13rMA2mbk9DPaf0f4VgQ0NUk5iOYf8i847svNAWyzixvuoe03t42q_zu-Gd6oBC6av9RR6GaYge2me2MUVqc_uYyFvtANnJ2EXtYdq73ObAM4AkL4RphHNN_0IKZesOnFhWKRp9JugyehcfpNc4mpvcnZZA1UlMNSTrLrAC-g441Uix_rRfx7ZQlIT4I2tEJHa_FeXEt5NAtczg7Il9p9-jgEMGtiPBTcm16OD_k6-dbLMbgQC6dok3c1fp-asF2YbVKY5SPTsX8ISBQcbu9vzhIVb6vYbQzLDSmsQ8RG7SWQ-1oAJurjVkGNjh6-Qkcu4SRMVein3PxSiHlWbEkfvcBIiQQv4tvEh7TIOaKcbMRZ0&refId=KB3iwr%2FES9gTgxjmGZbXkQ%3D%3D&trackingId=UD5OLV%2BsP7yFNnJW%2BOrVpw%3D%3D&trk=flagship3_search_srp_jobs'}


In [537]:
# Step 3: find the page 2 and 3 and 4 and iterate iteratively go through untill the very last apge. 


In [538]:
# Step 3: find the page 2 and 3 and 4 and iterate iteratively go through untill the very last apge. 
next_page_num = 1
while  True:
    # find the current page
    current_page_xpath = "//button[@aria-current='true']"
    current_page = int(browser.find_element("xpath", current_page_xpath).text)
    # find the next page number
    next_page_num = current_page+1
    # find it on the webpage
    next_page_xpath = "//button[@aria-label='Page "+str(next_page_num)+"']"
    try:
        next_page = browser.find_element("xpath", next_page_xpath)
        # next_page = WebDriverWait(browser, 10).until(EC.element_to_be_clickable((By.XPATH, next_page_xpath)))
        print ("loading page - " + str(next_page.text))
        # go to the next page
        next_page.click()
        # wait for the page to load
        time.sleep(3)
    except :
        print ('scanned all the pages')
        break

loading page - 2
loading page - 3
loading page - 4
loading page - 5
loading page - 6
loading page - 7
loading page - 8
loading page - …
loading page - 10
loading page - 11
loading page - 12
loading page - 13
loading page - 14
loading page - 15
loading page - 16
loading page - 17
loading page - 18
loading page - 19
loading page - 20
loading page - 21
loading page - 22
loading page - 23
loading page - 24
loading page - 25
loading page - 26
loading page - 27
loading page - 28
loading page - 29
loading page - 30
loading page - 31
loading page - 32
loading page - 33
loading page - 34
loading page - 35
loading page - 36
loading page - 37
loading page - 38
loading page - 39
loading page - 40
scanned all the pages


In [539]:
pip install pandas

Note: you may need to restart the kernel to use updated packages.


In [540]:

#import pandas as pd

#df = pd.DataFrame(table)

In [541]:
num=24


job_title = check_element (target_li_elements[num].find('a', class_=lambda x: x and 'title' in x))
company = check_element ( target_li_elements[num].find('span', class_= 'job-card-container__primary-description') )
location = check_element (target_li_elements[num].find('div', class_= 'artdeco-entity-lockup__caption ember-view') )
salary = check_element (target_li_elements[num].find('div', class_= lambda x: x and 'mt1 t-sans t-12 t-black--light' in x) )
link = target_li_elements[num].find('a').get('href')

dic = {     'job_title': job_title ,
            'company': company,
            'location': location,
            'salary_string': salary,
            'lower_bound_extracted_salary':  (extract_hourly_or_salary_range(salary)[0]),
            'upper_extracted_salary':  (extract_hourly_or_salary_range(salary)[1]),
            'average_extracted_salary':  (extract_hourly_or_salary_range(salary)[2]),
            'link': 'www.linkedin.com'+link
       
          }

print(dic)



{'job_title': 'Data Analyst', 'company': 'Insight Global', 'location': 'Charlotte, NC (Hybrid)', 'salary_string': '$60.56/hr', 'lower_bound_extracted_salary': 125964.8, 'upper_extracted_salary': 125964.8, 'average_extracted_salary': 125964.8, 'link': 'www.linkedin.com/jobs/view/3852177580/?eBP=CwEAAAGOZHBY-Zj1rel8Hjy_3Kl1K9Dn_IdcwkMEoFvSUUemAnEuWQzy2x1HG1ju-qHgxsKfXvIrvKLecrvfVVdxZFNJZqH13mZ-7NjCSLMaHgTb8utYtzpYr9kDx_eq5ZzK7sdfZ1pj7CVTbJZn-KGVzxh0B8I2blMBhtmzXmQ_5VMa8Q1BF6IGbN6EXnVs9y39yHvNPN6vQBVxLUwulneq82QpNJktz9onPICLo0NvaG4-_KWlHKWNyT58FReAa1_SfQaHwtq63DgD7aD2LnPeTdbIul9JputuOmaWGp6HqOGAQcZd6mBCregVGXzuTvxlVBxzsRHjNdkgYHWi1R8GELAW47wqZrtl4DE757BXAUa6RttcMLRfhf-D8vFNAg9s1R1h3SRaTADYzfeU-kFX7GvoU8gqIgI&refId=KB3iwr%2FES9gTgxjmGZbXkQ%3D%3D&trackingId=kLD4e9c7ADAkW9jozFmE%2Fg%3D%3D&trk=flagship3_search_srp_jobs'}


In [542]:
len (target_li_elements)

25

In [543]:
"""#def scrape_job_data_for_page(target_li_elements):
   # jobs_data = []

     #for num in range(len(target_li_elements)):
       #  job_title = check_element(target_li_elements[num].find('span', class_='job-details-jobs-unified-top-card_job-title-link'))         
         #location = check_element(target_li_elements[num].find('div', class_='jobs-details-jobs-unified-top-card_primary-description-without-tagline mbs')) 
  #       salary = check_element(target_li_elements[num].find('li', class_='jobs-details-jobs-unified-top-card_job-details-jobs-unified-top-card_job-insight-highlight'))
   
    #     link = target_li_elements[num].find('a').get('href')

      #   try:
            lower_bound, upper_bound, average = extract_hourly_or_salary_range(salary)
        # except Exception as e:
       #      print(f"Error extracting salary from '{salary}': {e}")
         #    lower_bound, upper_bound, average = None, None, None

   #      dic = {
         'job_title': job_title,
         'company': company,
            'location': location,
            'salary_string': salary,
            'lower_bound_extracted_salary': lower_bound,
            'upper_extracted_salary': upper_bound,
            'average_extracted_salary': average,
            'link': 'www.linkedin.com' + link
        }

        jobs_data.append(dic)

 return jobs_data

# Example function to fetch target_li_elements for a given page number
def get_target_li_elements_for_page(page_number):
    # This function should fetch and return the target li elements for a given page number
    # Implement this function according to your web scraping setup
    pass

# Example code to scrape job data from multiple pages
total_pages = 40  # Example total number of pages to scrape

for page_number in range(1, total_pages + 1):
    target_li_elements = get_target_li_elements_for_page(page_number)
    if target_li_elements:
        page_jobs_data = scrape_job_data_for_page(target_li_elements)
        jobs_data.append(page_jobs_data)

print("Total jobs data collected:", len(jobs_data))"""


'#def scrape_job_data_for_page(target_li_elements):\n   # jobs_data = []\n\n     #for num in range(len(target_li_elements)):\n       #  job_title = check_element(target_li_elements[num].find(\'span\', class_=\'job-details-jobs-unified-top-card_job-title-link\'))         \n         #location = check_element(target_li_elements[num].find(\'div\', class_=\'jobs-details-jobs-unified-top-card_primary-description-without-tagline mbs\')) \n  #       salary = check_element(target_li_elements[num].find(\'li\', class_=\'jobs-details-jobs-unified-top-card_job-details-jobs-unified-top-card_job-insight-highlight\'))\n   \n    #     link = target_li_elements[num].find(\'a\').get(\'href\')\n\n      #   try:\n            lower_bound, upper_bound, average = extract_hourly_or_salary_range(salary)\n        # except Exception as e:\n       #      print(f"Error extracting salary from \'{salary}\': {e}")\n         #    lower_bound, upper_bound, average = None, None, None\n\n   #      dic = {\n         \'job_

In [544]:
jobs_data = []
for num in range (0,len (target_li_elements)-1):
    job_title = check_element (target_li_elements[num].find('a', class_=lambda x: x and 'title' in x))
    company = check_element ( target_li_elements[num].find('span', class_= 'job-card-container__primary-description') )
    location = check_element (target_li_elements[num].find('div', class_= 'artdeco-entity-lockup__caption ember-view') )
    salary = check_element (target_li_elements[num].find('div', class_= lambda x: x and 'mt1 t-sans t-12 t-black--light' in x) )
    link = target_li_elements[num].find('a').get('href')

    dic = {     'job_title': job_title ,
                'company': company,
                'location': location,
                'salary_string': salary,
                'lower_bound_extracted_salary':  (extract_hourly_or_salary_range(salary)[0]),
                'upper_extracted_salary':  (extract_hourly_or_salary_range(salary)[1]),
                'average_extracted_salary':  (extract_hourly_or_salary_range(salary)[2]),
                'link': 'www.linkedin.com'+link
        
            }
    jobs_data.append(dic)

print(len(jobs_data))

24
